# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
import os

print(os.uname())

# In the cloud environment?
if 'root' in os.environ['HOME']:
    UENVPATH = '/data/'
    !pip -q install /home/workspace/python

# In the standalone environment?
if 'ferenc' in os.environ['HOME']:
    UENVPATH = '/home/ferenc/Python/rl/udadrl/data/'


from unityagents import UnityEnvironment
import numpy as np

import pprint as pp

from collections import deque

from time import time

# Import the helper files
from utilities import get_time_string, print_elapsed_time

import pdb # Debug! Debug! Debug! Debug! Debug! Debug! 

posix.uname_result(sysname='Linux', nodename='Anubis-Linux', release='5.4.0-48-generic', version='#52-Ubuntu SMP Thu Sep 10 10:58:49 UTC 2020', machine='x86_64')


Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
print(get_time_string(sep=True), ' Notebook for Continuous Control started.')

# ONE Agent, Standalone
#UENVCHOICE = 'Reacher_Linux/Reacher.x86_64'

# TWENTY Agents, Standalone
UENVCHOICE = 'Reacher_Linux_20/Reacher.x86_64'

# ONE Agent, Cloud, No-Visuals 
#UENVCHOICE = 'Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64'

# TWENTY Agents, Cloud, No-Visuals 
#UENVCHOICE = 'Reacher_Linux_NoVis/Reacher.x86_64'

env = UnityEnvironment( file_name=os.path.join( UENVPATH, UENVCHOICE ) )




2020-10-11 / 22-11-16  Notebook for Continuous Control started.


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [4]:
# FA  DEBUG! DEBUG! DEBUG!
print(f'Observation Space: {brain.vector_observation_space_size} dimensions of {brain.vector_observation_space_type} type') 
print(f'Action Space: {brain.vector_action_space_size} dimensions of {brain.vector_observation_space_type} type')

Observation Space: 33 dimensions of continuous type
Action Space: 4 dimensions of continuous type


### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [6]:
# Should a run with random actions be perfomed first?
RANDOMRUN = True

# Some reminders about lambda functions
# Thx2: https://realpython.com/python-lambda/
# Thx2: https://thispointer.com/python-how-to-use-if-else-elif-in-lambda-functions/
# thx2: https://www.blog.pythonlibrary.org/2016/05/24/python-101-an-intro-to-benchmarking-your-code/
# thx2: https://github.com/udacity/deep-reinforcement-learning/blob/master/python/unityagents/brain.py

env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)

epc = 0
print(f'We have {num_agents} active agent' + ( lambda x: 's' if x>1 else '' )(num_agents) )

start_time = time()
setup = "from unityagents import UnityEnvironment"

while RANDOMRUN:
    epc += 1
    #print(f'\r#{epc}', end = ' ')
    print(f'\r#{str(epc).rjust(6)}', end = ' ')
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    #pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! 
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    rewards = env_info.rewards
    nprewards = np.array(rewards)
    scores += rewards                         # update the score (for each agent)
    if np.sum(nprewards) > 0:
        maskagent = nprewards > 0
        agentbefore = False
        for (nagent, action) in enumerate(actions):
            if maskagent[nagent]:
                if agentbefore:
                    print('\r#' + '&'.rjust(6), end = ' ')
                print(' -> Agent {:0>2d} got reward {:+.5f} for action: {}'.format(nagent+1, rewards[nagent], action))
                agentbefore = True
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
        
end_time = time()

print_elapsed_time(start_time, end_time, "for choosing random actions")

print('#')
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

We have 20 active agents
#   168  -> Agent 01 got reward +0.02000 for action: [ 0.37321342 -0.15193966  0.24458722 -0.98354497]
#   169  -> Agent 01 got reward +0.04000 for action: [ 0.84749578  0.11950806 -0.00215177 -0.24673769]
#   170  -> Agent 01 got reward +0.04000 for action: [0.48241892 1.         0.47502348 0.07226327]
#   171  -> Agent 01 got reward +0.04000 for action: [ 0.84240164 -0.96279565 -1.          0.86088838]
#   172  -> Agent 01 got reward +0.04000 for action: [-0.5990983   0.64913198 -1.          0.22749427]
#   423  -> Agent 11 got reward +0.01000 for action: [-0.88119233  0.37612272  0.45096844 -0.27609803]
#   933  -> Agent 15 got reward +0.01000 for action: [ 0.33081275 -0.66025364 -0.52446978 -0.06126173]
#   934  -> Agent 15 got reward +0.04000 for action: [1.         1.         0.93020389 1.        ]
#   935  -> Agent 15 got reward +0.04000 for action: [-0.79087515 -0.86967656  0.95560411 -0.44126424]
#   936  -> Agent 15 got reward +0.04000 for action: [-0

### Re check running time (Random run)
This time with the leanest code possible

In [7]:
# Should another run with random actions be perfomed?
RANDOMRUN = True

env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)

epc = 0
print(f'We have {num_agents} active agent' + ( lambda x: 's' if x>1 else '' )(num_agents) )

start_time = time()

while RANDOMRUN:
    epc += 1
    print(f'\r#{str(epc).rjust(6)}', end = ' ')
    #pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! 
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    dones = env_info.local_done                        # see if episode finished
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
        
end_time = time()

print_elapsed_time(start_time, end_time, "for choosing fast random actions")

print('#')
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

We have 20 active agents
#  1001 
***--> Total Elapsed Runtime: 00:00:02 for choosing fast random actions
#
Total score (averaged over agents) this episode: 0.0


### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

##### FA: One BIG MISUNDERSTANDING & STACKS OF TENSORS (23-08-2020) --> #FA; BMSoT:
It has cost me several days if not weeks to get behind the fact that the [A2C sample implementation of Miguel](https://github.com/mimoralea/gdrl/blob/master/notebooks/chapter_11/chapter-11.ipynb) is working with **stacks of tensors** instead of single tensors. Which was especially hard to find because the PyTorch code looks exactly the same for both.

In the end, when I thought about it, it makes sense and is a nifty feature of PyTorch. It is just not obvious to people like me, without in deep insights in the inner workings of PyTorch. 

Furthermore the authors of the PyTorch documentation seem not to make it too visible, I had to dig it out of one of the function definitions I use, however inderectly over the layer defintion for a2cnet:

https://pytorch.org/docs/0.4.0/_modules/torch/nn/functional.html#linear

> def linear(input, weight, bias=None):
>    """
>    Applies a linear transformation to the incoming data: :math:`y = xA^T + b`.
>
>    Shape:
>        - Input: :math:`(N, *, in\_features)` where `*` means any number of
>          additional dimensions
>        - Weight: :math:`(out\_features, in\_features)`
>        - Bias: :math:`(out\_features)`
>        - Output: :math:`(N, *, out\_features)`
>    """
>    if input.dim() == 2 and bias is not None:
>        # fused op is marginally faster
>        return torch.addmm(bias, input, weight.t())
>
>    output = input.matmul(weight.t())
>    if bias is not None:
>        output += bias
>    return output


## Train the agent

In [8]:
import mya2cagent

# Thx2: https://emacs.stackexchange.com/a/13483
import imp
imp.reload(mya2cagent)

agent = mya2cagent.a2cagent(len(env_info.agents), env, brain, max_steps = 30000, max_n_steps = 65\
                            , policy_loss_weight = 0.99, entropy_loss_weight = 0.001, value_loss_weight = 0.1)

In [9]:
# Thx2: https://discuss.pytorch.org/t/how-to-check-if-model-is-on-cuda/180/2?u=ferenc_acs
start_time = time()

#agent.train(gamma = 0.99, tau = 0.95)
agent.train(gamma = 0.4, tau = 0.95)

end_time = time()

print_elapsed_time(start_time, end_time, "for training the agent")

Training iteration: 1000          last optimization: 975# --> Environment reset at iteration: 1001
Training iteration: 2001         last optimization: 1976+ --> Environment reset at iteration: 2002
Training iteration: 3002         last optimization: 2977# --> Environment reset at iteration: 3003
Training iteration: 4003         last optimization: 3978+ --> Environment reset at iteration: 4004
Training iteration: 5004         last optimization: 4979# --> Environment reset at iteration: 5005
Training iteration: 6005         last optimization: 5980+ --> Environment reset at iteration: 6006
Training iteration: 7006         last optimization: 6981# --> Environment reset at iteration: 7007
Training iteration: 8007         last optimization: 7982+ --> Environment reset at iteration: 8008
Training iteration: 9008         last optimization: 8983# --> Environment reset at iteration: 9009
Training iteration: 10009         last optimization: 9984+ --> Environment reset at iteration: 10010
Training

In [15]:
# Some reminders about lambda functions
# Thx2: https://realpython.com/python-lambda/
# Thx2: https://thispointer.com/python-how-to-use-if-else-elif-in-lambda-functions/
import torch


TESTRUN = True


env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
#env_info = env.reset(train_mode=True)[brain_name] # Debug! Debug! Debug! Debug! Debug! Debug! 
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
avg100sum = np.zeros(num_agents)
exectime = 0

scores100 = deque(avg100sum, 100)
time100 = list()
epc = 0


#import pdb; pdb.set_trace() # Debug! Debug! Debug! Debug! Debug! Debug! 
start_time = time()

with torch.no_grad():
    while TESTRUN: #== True:
        epc += 1
        #print(f'\r#{epc}', end = ' ')
        print(f'\r#{str(epc).rjust(6)}', end = ' ')
        #actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = agent.a2c_net.select_action(states)
        #actions = np.clip(actions.detach().cpu().numpy(), -1, 1)                  # all actions between -1 and 1

        t_step_b = time()
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        t_step_e = time()
        time100.append(t_step_e - t_step_b)

        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        rewards = env_info.rewards
        nprewards = np.array(rewards)
        scores += rewards                         # update the score (for each agent)
        scores100.append(rewards)
        if np.sum(nprewards) > 0:
            #for x in scores100:
            #    avg100sum += x
            #print(f'\r#{epc} Avg 100 Rewards = {np.mean(scores100)}')  
            avg100sum = np.zeros(num_agents)
            maskagent = nprewards > 0

        if epc%100 == 0:
            print(f'Exec time Avg 100: {np.mean(time100)}')
            time100 = []

        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
        
end_time = time()

print_elapsed_time(start_time, end_time, "for observing the trained agent")
        
print('#')
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

#   100 Exec time Avg 100: 0.094151451587677
#   200 Exec time Avg 100: 0.09807056903839112
#   300 Exec time Avg 100: 0.09465308427810669
#   400 Exec time Avg 100: 0.0946094274520874
#   500 Exec time Avg 100: 0.0980296230316162
#   600 Exec time Avg 100: 0.09466694593429566
#   700 Exec time Avg 100: 0.09801042079925537
#   800 Exec time Avg 100: 0.09810897827148438
#   900 Exec time Avg 100: 0.09462484836578369
#  1000 Exec time Avg 100: 0.10143831491470337
#  1001 
***--> Total Elapsed Runtime: 00:01:38 for observing the trained agent
#
Total score (averaged over agents) this episode: 0.5619999874383211


In [16]:
env.close()

In [17]:
print(get_time_string(sep=True), ' Notebook for Continuous Control ended.')

2020-10-11 / 22-24-08  Notebook for Continuous Control ended.
